# Python from container

For me it's common task to build interaption with container which handle python program.

So in this section I want to show you how to build such an interaction.

## Python image

You need to create a Python image. The following cell describes the docker file that's used to create the image.

In [1]:
%%writefile python_container/dockerfile
FROM python:3.11.4
WORKDIR program
COPY script.py script.py
COPY requirements.txt requirements.txt
RUN pip3 install -r requirements.txt

Overwriting python_container/dockerfile


The next cell describes the `requirements.txt` we'll need for this container.

In [2]:
%%writefile python_container/requirements.txt
numpy==1.24.2
psycopg2-binary==2.9.6

Overwriting python_container/requirements.txt


Finally, we build the image.

In [3]:
%%bash
docker build -t pg_example_python python_container &> /dev/null

## Start containers

We need to create at least one table in our postgres database. So in the following cell we store the script that will be used to initialise the database.

In [4]:
%%writefile python_container/create_table.sql
CREATE TABLE main_table(
    id TEXT NOT NULL,
    text TEXT NOT NULL
);

Writing python_container/create_table.sql


Next, start the `python` and `postgres` containers.

You should share the same net between containers, in the following example which is completed by `test_project_net`.

In [5]:
%%bash
# network
docker network create test_project_net &> /dev/null
# postgres
docker run --rm -d\
    --name pg_example_posgres_cont\
    -e POSTGRES_USER=docker_app\
    -e POSTGRES_PASSWORD=docker_app\
    -e POSTGRES_DB=docker_app_db\
    --net=test_project_net\
    -v ./python_container/create_table.sql:/docker-entrypoint-initdb.d/create_table.sql\
    postgres:15.4 &> /dev/null
# python
docker run --rm -itd\
    --name pg_example_python_cont\
    --net=test_project_net\
    pg_example_python &> /dev/null

## Executing python

In the next cell, we run the Python script in the Python container. The script simply adds some random values to the `main_table` from the **ohter container**. The message "Adding records is done!" indicates that the Python program was executed normally.

In [7]:
%%bash
docker exec pg_example_python_cont bash -c "python3 script.py"

Adding records is done!


## Check the table

So now, to make sure we have done everything right, let us select values from the created table.

In [8]:
%%bash
docker exec pg_example_posgres_cont bash -c \
"psql -U docker_app -d docker_app_db -c \"SELECT * FROM main_table;\""

 id |         text         
----+----------------------
 0  | rslmljobfthxylvqmxjx
 1  | bfrtjjwbrmvmezusqqbt
 2  | opujcieiufdicgkkerup
 3  | lnuczeqsgxuqxeqfklva
 4  | sgmbhqliwlidpgpuguyh
 5  | cmotqncbzehqgovnmain
 6  | xnrkwzjcnwsfqzmgnnol
 7  | hjiihyoenlmxeydsdcnc
 8  | bdrsfcxsbwzxieiityah
 9  | dozhyfgltysgubaekvca
 10 | yyjtebgjeiryoozwnliq
 11 | httaabixdruuncanpkdx
 12 | fvilvakiasqmkpmoxxze
 13 | sitdcxqjzdmcrefldcjc
 14 | jlqyxrhsgxenzmmihtsc
 15 | rumlutiupquvfucxfmgn
 16 | arsdyzmlxfruodumuupk
 17 | jzjoyadntdjbcfucgsku
 18 | xiqpdchofpxsksjlmvwv
 19 | wuqfxhniyrusexnjrcin
(20 rows)



## Stop containers

In [9]:
%%bash
docker stop pg_example_posgres_cont pg_example_python_cont &> /dev/null
docker network rm test_project_net &> /dev/null

## Clear python image

To avoid creating a lot of rubbish in docker images, you should delete the container created for this example.

In [10]:
%%bash
docker rmi pg_example_python &> /dev/null